<div align="center">
 <img src="https://raw.githubusercontent.com/matheusmota/dataviz2018/master/resources/images/logo_facens_pos.png" width="100x">
    <h3>Aplicações de Aprendizado de Máquina e Processamento de Linguagem Natural</h3>  
    <h1><center>Frequent Itemset Mining</center></h1>
</div>


* **203066**  - Evandro Bertolucci
* **110257**  - João Victor Carvalho
* **203071**  - Louise Constantino
* **203087**  - Luiza Constantino
* **203019**  - Murilo Piva
* **203263**  - Rafael Henrique

Para este exercício exploramos os dados de uma competição no Kaggle, disponível no link abaixo:

https://www.kaggle.com/c/instacart-market-basket-analysis

A empresa Instacart fornece serviços de compra e entrega através de um App. Entre seus objetivos estão identificar quais produtos os clientes:

- Comprariam novamente;
- Tentariam usar pela primeira vez;
- Deixariam para a próxima compra;

Usaremos a base transacional fornecida para buscar regras de associação. A idéia é buscar produtos que são frequentemente comprados em conjunto para orientar estratégias de cross-selling.

Para não implementar o algoritmo do zero, a biblioteca MLxtend será aplicada. Segue abaixo o link com detalhes da sua instalação no ambiente Anaconda;

https://anaconda.org/conda-forge/mlxtend

## Carregando os dados

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import gc
import random
import sys

Carrega cestas de compra em um dataframe. Em função do volume de dados frequentemente usaremos estratégias para reduzir o uso de memória. Neste caso carregamos apenas as colunas indispensáveis: código da compra e do produto.

In [2]:
url = r"https://raw.githubusercontent.com/brvnl/AplicacoesAprendizadoMaquina/main/order_products__train.csv"
df_raw = pd.read_csv(url, usecols=["order_id", "product_id"])

Checando o número de linhas, colunas e a memória ocupada:

In [3]:
df_raw.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1384617 entries, 0 to 1384616
Data columns (total 2 columns):
 #   Column      Non-Null Count    Dtype
---  ------      --------------    -----
 0   order_id    1384617 non-null  int64
 1   product_id  1384617 non-null  int64
dtypes: int64(2)
memory usage: 21.1 MB


Analisando uma amostra dos dados:

In [4]:
df_raw.head(3)

,order_id,product_id
0,1,49302
1,1,11109
2,1,10246


Verificando carcaterísticas do dataset:

In [5]:
tt_compras = len(df_raw["order_id"].unique())
tt_produtos = len(df_raw["product_id"].unique())

print("Quantas compras diferentes? %d." %(tt_compras))
print("Quantos produtos diferentes? %d." %(tt_produtos))

Quantas compras diferentes? 131209.
Quantos produtos diferentes? 39123.


Carregando tabela com nomes dos produtos e convertendo em um dicionário:

In [6]:
url_p = r"https://raw.githubusercontent.com/brvnl/AplicacoesAprendizadoMaquina/main/products.csv"
df_products_lookup = pd.read_csv(url_p)

d = dict(zip(df_products_lookup["product_id"], df_products_lookup["product_name"]))
del df_products_lookup

# Filtrando os dados

Por questão de uso de memória, pode ser necessário executar a análise por partes. Para isso, o código abaixo permite filtrar apenas compras/produtos de uma ilha ou departamento.

In [7]:
def filter_aisle_department(df, aisle = None, department = None):
    url = r"https://github.com/brvnl/AplicacoesAprendizadoMaquina/blob/main/product_aisle_department_lookup.xlsx?raw=true"
    
    if aisle is not None:
        dft = pd.read_excel(url, sheet_name="Products", usecols=["product_id", "aisle"])
        dft = pd.merge(df, dft, left_on="product_id", right_on="product_id")
        dft = dft[dft["aisle"].isin(aisle)]
        dft.drop(columns="aisle", inplace=True)
        
    elif department is not None:
        dft = pd.read_excel(url, sheet_name="Products", usecols=["product_id", "department"])
        dft = pd.merge(df, dft, left_on="product_id", right_on="product_id")
        dft = dft[dft["department"].isin(department)]
        dft.drop(columns="department", inplace=True)
    
    else:
        dft = df
    
    compras = len(dft["order_id"].unique())
    produtos = len(dft["product_id"].unique())
    particip = produtos / tt_produtos * 100
    
    print("Compras diferentes: %d" %(compras))
    print("Produtos diferentes: %d" %(produtos))
    print(f'Participação nas vendas: {particip:.2f}%')
    return dft

- Os parametros aisle e department devem ser sempre **None** ou na forma de lista (mesmo com um único elemento). 
- Na implementação atual o fitro ocorre ou por ilha ou por departamento, mas não junto. Adapte o código caso deseje o filtor duplo.


Função para Resultados, trocando os códigos dos produtos por sua descrição:

In [8]:
def int_from_frozenset(row):
    return d[list(row)[0]]

Carregando departamentos e ilhas:

In [9]:
url_filtros = r"https://github.com/brvnl/AplicacoesAprendizadoMaquina/blob/main/product_aisle_department_lookup.xlsx?raw=true"

df_aisles = pd.read_excel(url_filtros, sheet_name="Aisle")
df_departs = pd.read_excel(url_filtros, sheet_name="Departments")

## Automatizar filtros

In [10]:
def mining(tp_filtro, combinacoes, total_compras, df_original):
    df_selected = pd.DataFrame(columns=['antecedents','consequents','antecedent support','consequent support','support','confidence','lift','leverage','conviction','antecedents_name','consequents_name','antecedents_particip(%)','consequents_particip(%)','filters'])
    
    #Carregando departamentos e ilhas:
    url_filtros = r"https://github.com/brvnl/AplicacoesAprendizadoMaquina/blob/main/product_aisle_department_lookup.xlsx?raw=true"

    df_aisles = pd.read_excel(url_filtros, sheet_name="Aisle")
    df_departs = pd.read_excel(url_filtros, sheet_name="Departments")
    cAisles = len(df_aisles)
    cDeparts = len(df_departs)
    
    
    cont = 0
    
    while cont < combinacoes:
        try:
            #Busca randomicamente departamentos/ilhas para filtragem
            if tp_filtro == 'departments':
                filters = df_departs[df_departs["department_id"].isin(random.sample(range(1, cDeparts), 2))]["department"].tolist()
                print("\nDepartamentos selecionados:",filters)  
                df_raw = filter_aisle_department(df_original, aisle = None, department = filters) 

            elif tp_filtro == 'aisles':
                filters = df_aisles[df_aisles["aisle_id"].isin(random.sample(range(1, cAisles), 2))]["aisle"].tolist()
                print("\nIlhas selecionadas:",filters)  
                df_raw = filter_aisle_department(df_original, aisle = filters, department = None) 

    
            # ================ Ajustando o layout ================
            df_raw["comprado"] = 1

            df1 = df_raw.pivot_table(index='order_id', columns='product_id', values="comprado", fill_value=0)



            # ============ Buscando regras com APRIORI ============

            # Acha os conjuntos frequentes que atendem um suporte minimo (%)        
            frequent_itemsets = apriori(df1, min_support=0.005)

            # Filtra os conjuntos pela métrica confiança
            if len(frequent_itemsets):
                selected = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.02)

            #Apresentando os resultados, trocando os códigos dos produtos por sua descrição.    
            if len(frequent_itemsets):
                selected["antecedents_name"] = selected["antecedents"].apply(int_from_frozenset)
                selected["consequents_name"] = selected["consequents"].apply(int_from_frozenset)    

                #Verificando a % de participação do produto em relação ao total de compras
                for index, row in selected.iterrows(): 
                    ant = len(df_original[df_original['product_id'] == list(row['antecedents'])[0]].order_id.unique())
                    cons = len(df_original[df_original['product_id'] == list(row['consequents'])[0]].order_id.unique())
                    selected.loc[index,"antecedents_particip(%)"] = ant / total_compras * 100
                    selected.loc[index,"consequents_particip(%)"] = cons / total_compras * 100
                                        
                    selected.loc[index,"filters"] = tp_filtro +': '+ ' '.join(filters)


                #Armazena os resultados da iteração
                df_selected = pd.concat([df_selected,selected], ignore_index=True) 
            
            if not selected.empty:
                cont += 1   
            else
                print("Não há itemsets que atendam ao mínimo suporte fornecido.")
            
            #Desalocar memória
            gc.collect()
        except:
            pass
    
    return df_selected
    

## Executando Iteração - Departamentos

In [11]:
result_deps = mining('departments', 5, tt_compras, df_raw)

Departamentos selecionados: ['other', 'pets']
Compras diferentes: 4399
Produtos diferentes: 1128
Participação nas vendas: 2.88%

Departamentos selecionados: ['bulk', 'personal care']
Compras diferentes: 16093
Produtos diferentes: 4349
Participação nas vendas: 11.12%

Departamentos selecionados: ['bakery', 'international']
Compras diferentes: 42309
Produtos diferentes: 2196
Participação nas vendas: 5.61%

Departamentos selecionados: ['bulk', 'meat seafood']
Compras diferentes: 25339
Produtos diferentes: 812
Participação nas vendas: 2.08%

Departamentos selecionados: ['international', 'deli']
Compras diferentes: 38493
Produtos diferentes: 2002
Participação nas vendas: 5.12%

Departamentos selecionados: ['pets', 'pantry']
Compras diferentes: 49163
Produtos diferentes: 4862
Participação nas vendas: 12.43%

Departamentos selecionados: ['international', 'deli']
Compras diferentes: 38493
Produtos diferentes: 2002
Participação nas vendas: 5.12%

Departamentos selecionados: ['other', 'deli']
Co

## Executando Iteração - Ihas

In [12]:
result_ilhas = mining('aisles', 5, tt_compras, df_raw)

Ilhas selecionados: ['specialty cheeses', 'marinades meat preparation']
Compras diferentes: 6183
Produtos diferentes: 518
Participação nas vendas: 1.32%

Ilhas selecionados: ['marinades meat preparation', 'prepared meals']
Compras diferentes: 6289
Produtos diferentes: 561
Participação nas vendas: 1.43%

Ilhas selecionados: ['prepared soups salads', 'kitchen supplies']
Compras diferentes: 3034
Produtos diferentes: 240
Participação nas vendas: 0.61%

1
Ilhas selecionados: ['prepared soups salads', 'tofu meat alternatives']
Compras diferentes: 6683
Produtos diferentes: 256
Participação nas vendas: 0.65%

Ilhas selecionados: ['tofu meat alternatives', 'baking ingredients']
Compras diferentes: 13674
Produtos diferentes: 656
Participação nas vendas: 1.68%

Ilhas selecionados: ['prepared meals', 'packaged seafood']
Compras diferentes: 4530
Produtos diferentes: 325
Participação nas vendas: 0.83%

Ilhas selecionados: ['baking ingredients', 'bulk dried fruits vegetables']
Compras diferentes: 106

## Relações encontradas

Descreva no campo abaixo as relações mais interessantes encontradas e como você chegou a elas.

In [38]:
result_deps.sort_values(['confidence','antecedents_particip(%)'],ascending=False).head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_name,consequents_name,antecedents_particip(%),consequents_particip(%),filters
1090,"(2178, 1287)",(624),0.010198,0.154915,0.005314,0.521099,3.363782,0.003734,1.764635,Natural Deli Slices Gouda Cheese,Chub Chorizo,0.008384,0.003811,departments: produce meat seafood
349,"(5347, 3143)",(1493),0.009663,0.146798,0.005036,0.521099,3.549769,0.003617,1.781584,"Adults MultiGummies, Cherry, Berry, Orange",Powder Bath With Hawaiian Kukui Oil Eucalyptus...,0.000762,0.000000,departments: produce pantry
1084,"(1000, 2178)",(624),0.015401,0.154915,0.007105,0.461339,2.978018,0.004719,1.568862,Apricots,Chub Chorizo,0.066306,0.003811,departments: produce meat seafood
698,"(2144, 4703)",(1384),0.014419,0.145033,0.006652,0.461339,3.180911,0.004561,1.587206,Doritos,Mang-O-Rita Margarita With A Twist,0.026675,0.000000,departments: frozen produce
342,"(2410, 5347)",(1493),0.014594,0.146798,0.006733,0.461339,3.142675,0.004591,1.583930,Original Scent Liquid Fabric Softener,Powder Bath With Hawaiian Kukui Oil Eucalyptus...,0.003049,0.000000,departments: produce pantry


In [39]:
result_ilhas.sort_values(['confidence','antecedents_particip(%)'],ascending=False).head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_name,consequents_name,antecedents_particip(%),consequents_particip(%),filters
62,(547),(766),0.013845,0.019726,0.006327,0.456989,23.166768,0.006054,1.805257,1000 Roses Floral Toner for Sensitive Skin,Refried Beans Vegetarian,0.000000,0.022102,aisles: energy granola bars oral hygiene
65,(606),(754),0.014292,0.018758,0.005806,0.406250,21.656994,0.005538,1.652617,Salted Caramel Ice Cream,Hydro Silk Women's Sensitive Skin Refill Blades,0.003811,0.002286,aisles: energy granola bars oral hygiene
63,(766),(547),0.019726,0.013845,0.006327,0.320755,23.166768,0.006054,1.451839,Refried Beans Vegetarian,1000 Roses Floral Toner for Sensitive Skin,0.022102,0.000000,aisles: energy granola bars oral hygiene
64,(754),(606),0.018758,0.014292,0.005806,0.309524,21.656994,0.005538,1.427577,Hydro Silk Women's Sensitive Skin Refill Blades,Salted Caramel Ice Cream,0.002286,0.003811,aisles: energy granola bars oral hygiene
1,(127),(143),0.019776,0.026038,0.005274,0.266667,10.241350,0.004759,1.328130,Marscapone,Organic Lemons,0.018291,0.002286,aisles: prepared soups salads kitchen supplies


## Relações encontradas

Descreva no campo abaixo as relações mais interessantes encontradas e como você chegou a elas.

<div>
    <h2><b>Análise 1</b></h2>  
    <br>
    <li>Filtro: departments = ["produce meat", "seafood"]</li>
    <li>min_support=0.005</li>
    <li>metric="confidence", min_threshold=0.02</li>
    <p><b>Regras:</b></p>
    <br>
    <li>Regra 1: Natural Deli Slices Gouda Cheese → Chub Chorizo</li>  
    <li>Regra 2: Apricots → Chub Chorizo</li>
    <br>
    <br>
    <h2><b>Análise 2</b></h2>  
    <br>
    <li>Filtro: departments = ["produce", "pantry"]</li>
    <li>min_support=0.005</li>
    <li>metric="confidence", min_threshold=0.02</li>
    <p><b>Regras:</b></p>
    <br>
    <li>Regra 1: Adults MultiGummies, Cherry, Berry, Orange → Powder Bath With Hawaiian Kukui Oil Eucalyptus</li>
    <li>Regra 2: Original Scent Liquid Fabric Softener → Powder Bath With Hawaiian Kukui Oil Eucalyptus</li>
    <br>
    <br>
    <h2><b>Análise 3</b></h2>  
    <br>
    <li>Filtro: departments = ["frozen", "produce"]</li>
    <li>min_support=0.005</li>
    <li>metric="confidence", min_threshold=0.02</li>
    <p><b>Regras:</b></p>
    <br>
    <li>Regra 1: Doritos → Mang-O-Rita Margarita With A Twist</li>
    <br>
    <br>
    <br>
    <h5>Observação: </h5> Não consideramos as análises por Ilha, pois por Departamentos identificamos Regras melhores. 
</div>